In [8]:
import pandas as pd
import folium
import numpy as np
import requests

#. Function for generating folium.map object
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [9]:
base_map = generateBaseMap()


In [4]:
# import NYC taxi driver data
df_train = pd.read_csv('input/train.csv').drop(columns=['trip_duration', 'dropoff_datetime'])
df_test = pd.read_csv('input/test.csv')
df = pd.concat([df_train, df_test], sort=False, ignore_index=True)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083778 entries, 0 to 2083777
Data columns (total 9 columns):
id                    object
vendor_id             int64
pickup_datetime       object
passenger_count       int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag    object
dtypes: float64(4), int64(2), object(3)
memory usage: 143.1+ MB


In [11]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime, format='%Y-%m-%d %H:%M:%S')
df['month'] = df.pickup_datetime.apply(lambda x: x.month)
df['week'] = df.pickup_datetime.apply(lambda x: x.week)
df['day'] = df.pickup_datetime.apply(lambda x: x.day)
df['hour'] = df.pickup_datetime.apply(lambda x: x.hour)
df.head()
#df.info()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,month,week,day,hour
0,id2875421,2,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,N,3,11,14,17
1,id2377394,1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,N,6,23,12,0
2,id3858529,2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,N,1,3,19,11
3,id3504673,2,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,N,4,14,6,19
4,id2181028,2,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,N,3,12,26,13


In [12]:
eg_loc = df.loc[1,"pickup_longitude":"dropoff_latitude"]
eg_pickup_log = eg_loc[0]
eg_pickup_lat = eg_loc[1]
eg_dropoff_log = eg_loc[2]
eg_dropoff_lat = eg_loc[3]
print(eg_loc)

#sub_df = df.loc[1,"pickup_longitude":"pickup_latitude"]
#sample_pickup_loc = np.array([[ sub_df[0],   sub_df[1]]])
#sample_pickup = pd.DataFrame(sample_pickup_loc, columns = ['sample_pick_long', 'sample_pick_lat'])

pickup_longitude    -73.9804
pickup_latitude      40.7386
dropoff_longitude   -73.9995
dropoff_latitude     40.7312
Name: 1, dtype: object


In [10]:
# Display the sample pickup location
radius_ = 20
# Add pickup location
folium.CircleMarker(location=[eg_pickup_lat, eg_pickup_log], radius=radius_,
                    color='red',
                    fill_color='#3186cc').add_to(base_map)
folium.Marker([eg_pickup_lat, eg_pickup_log],icon=folium.Icon(color='blue')).add_to(base_map)
#. Add dropoff location
folium.CircleMarker(location=[eg_dropoff_lat, eg_dropoff_log], radius=radius_,
                    color='#F08080',
                    fill_color='#3186cc').add_to(base_map)
folium.Marker([eg_dropoff_lat, eg_dropoff_log],icon=folium.Icon(color='green')).add_to(base_map)
base_map

In [ ]:
# Extracting the trajectory from the fastest route profile


In [7]:
#. Request fastest routes from OSRM server
osrm_route_req_str = 'http://router.project-osrm.org/route/v1/driving/'
current_req_route = osrm_route_req_str + str(eg_pickup_lat)+','+str(eg_pickup_log)+';'+str(eg_dropoff_lat)+','+str(eg_dropoff_log)+';0,0?overview=full'
print(current_req_route)

http://router.project-osrm.org/route/v1/driving/40.738563537597656,-73.98041534423827;40.73115158081055,-73.99948120117188;0,0?overview=full


In [12]:
#fastest_route = requests.get('http://router.project-osrm.org/route/v1/driving/13.388860,52.517037;13.397634,52.529407;13.428555,52.523219?overview=full')
fastest_route = requests.get(current_req_route)


In [13]:
print(fastest_route)
fastest_route.json()['routes']

<Response [429]>


KeyError: 'routes'